In [1]:
from pyiron_atomistics import Project

2024-06-24 10:42:14,981 - pyiron_log - WARNING - pyiron found a 'templates' folder in the /home/tlakshmi/pyiron/resources resource directory. These are no longer supported in pyiron_base >=0.7.0. They are replaced by Project.create_job_class() and Project.wrap_python_function().


In [2]:
#from getpass import getpass

In [3]:
#MISTRAL_API_KEY = getpass(prompt='Enter-api-key')

In [4]:
from langchain_core.messages import HumanMessage
from langchain_mistralai.chat_models import ChatMistralAI

In [5]:
from ase.build import bulk

In [6]:
from ase.data import reference_states, atomic_numbers

In [7]:
from langchain.agents import tool

In [8]:
@tool
def get_crystal_structure(chemical_symbol: str) -> str:
    """Returns the atomic crystal structure of a chemcial symbol"""
    ref_state = reference_states[atomic_numbers[chemical_symbol]]
    if ref_state is None:
        return "No crystal structure known."
    else:
        return ref_state["symmetry"]

In [9]:
get_crystal_structure.invoke("Fe")

'bcc'

In [10]:
get_crystal_structure.invoke("Au")

'fcc'

In [11]:
#from langchain_openai import ChatOpenAI

#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

In [28]:
llm = ChatMistralAI(api_key='key-here')

In [14]:
tools = [get_crystal_structure]

In [15]:
"""
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "You are very powerful assistant, but don't know current events.",  # This initial query fails when car is provided as input rather than gold
            "You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
"""

'\nfrom langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder\n\nprompt = ChatPromptTemplate.from_messages(\n    [\n        (\n            "system",\n            # "You are very powerful assistant, but don\'t know current events.",  # This initial query fails when car is provided as input rather than gold\n            "You are very powerful assistant, but don\'t know current events. For each query vailidate that it contains a chemical element and otherwise cancel.",\n        ),\n        ("user", "{input}"),\n        MessagesPlaceholder(variable_name="agent_scratchpad"),\n    ]\n)\n'

In [16]:
llm_with_tools = llm.bind_tools(tools)

In [17]:
"""
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
"""

'\nfrom langchain.agents.format_scratchpad.openai_tools import (\n    format_to_openai_tool_messages,\n)\nfrom langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser\n\nagent = (\n    {\n        "input": lambda x: x["input"],\n        "agent_scratchpad": lambda x: format_to_openai_tool_messages(\n            x["intermediate_steps"]\n        ),\n    }\n    | prompt\n    | llm_with_tools\n    | OpenAIToolsAgentOutputParser()\n)\n'

In [18]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)

In [19]:
from langchain.agents import initialize_agent
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    early_stopping_method="generate",
    memory=memory,
    handle_parsing_errors=True,
)

/home/tlakshmi/anaconda3/envs/pyiron_dataclasses_20240614/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [20]:
#from langchain.agents import AgentExecutor

#agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [21]:
#lst = list(agent_executor.stream({"input": "What is the crystal structure of gold"}))  # Yeah this worked !!

In [22]:
agent("You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. What is the crystal structure of Car")

/home/tlakshmi/anaconda3/envs/pyiron_dataclasses_20240614/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Parameter `stop` not yet supported (https://docs.mistral.ai/api)




> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "I'm sorry for any confusion, but the abbreviation 'Car' does not correspond to a chemical element. The crystal structure can only be determined for specific chemical elements. If you have a question about the crystal structure of a specific element, please let me know."
}
```

> Finished chain.


{'input': "You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. What is the crystal structure of Car",
 'chat_history': [],
 'output': "I'm sorry for any confusion, but the abbreviation 'Car' does not correspond to a chemical element. The crystal structure can only be determined for specific chemical elements. If you have a question about the crystal structure of a specific element, please let me know."}

In [23]:
agent("You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. What is the crystal structure of Gold")

Parameter `stop` not yet supported (https://docs.mistral.ai/api)




> Entering new AgentExecutor chain...
Here is my response:
```json
{
    "action": "get_crystal_structure",
    "action_input": "Gold"
}
```

KeyError: 'Gold'

In [24]:
agent("You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. What is the crystal structure of Au")

Parameter `stop` not yet supported (https://docs.mistral.ai/api)




> Entering new AgentExecutor chain...


Parameter `stop` not yet supported (https://docs.mistral.ai/api)


```json
{
    "action": "get_crystal_structure",
    "action_input": "Au"
}
```
Observation: fcc
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The crystal structure of gold (Au) is face-centered cubic (fcc)."
}
```

> Finished chain.


{'input': "You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. What is the crystal structure of Au",
 'chat_history': [HumanMessage(content="You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. What is the crystal structure of Car"),
  AIMessage(content="I'm sorry for any confusion, but the abbreviation 'Car' does not correspond to a chemical element. The crystal structure can only be determined for specific chemical elements. If you have a question about the crystal structure of a specific element, please let me know.")],
 'output': 'The crystal structure of gold (Au) is face-centered cubic (fcc).'}

In [25]:
agent("You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. What is the crystal structure of Na")

Parameter `stop` not yet supported (https://docs.mistral.ai/api)




> Entering new AgentExecutor chain...


Parameter `stop` not yet supported (https://docs.mistral.ai/api)


```json
{
    "action": "get_crystal_structure",
    "action_input": "Na"
}
```
Observation: bcc
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The crystal structure of sodium (Na) is body-centered cubic (bcc)."
}
```

> Finished chain.


{'input': "You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. What is the crystal structure of Na",
 'chat_history': [HumanMessage(content="You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. What is the crystal structure of Car"),
  AIMessage(content="I'm sorry for any confusion, but the abbreviation 'Car' does not correspond to a chemical element. The crystal structure can only be determined for specific chemical elements. If you have a question about the crystal structure of a specific element, please let me know."),
  HumanMessage(content="You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. What is the crystal structure of Au"),
  AIMessage(content='The crystal structure of gold (Au) is face-centered 

In [26]:
agent("You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. What is the crystal structure of Ti")

Parameter `stop` not yet supported (https://docs.mistral.ai/api)




> Entering new AgentExecutor chain...


Parameter `stop` not yet supported (https://docs.mistral.ai/api)


```json
{
    "action": "get_crystal_structure",
    "action_input": "Ti"
}
```
Observation: hcp
Thought:

HTTPStatusError: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}

In [27]:
agent("You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel. What is the crystal structure of Titanium")

Parameter `stop` not yet supported (https://docs.mistral.ai/api)




> Entering new AgentExecutor chain...


Parameter `stop` not yet supported (https://docs.mistral.ai/api)


Here is my response in the format you requested:

```json
{
    "action": "get_crystal_structure",
    "action_input": "Ti"
}
```

This response will allow the user to look up the crystal structure of titanium (Ti) using the specified tool.
Observation: hcp
Thought:

HTTPStatusError: Error response 429 while fetching https://api.mistral.ai/v1/chat/completions: {"message":"Requests rate limit exceeded"}